# CS6405 - Data Mining - First Assignment

Lecturer: Dr Andrea Visentin


## Submission

This assignment is **due on 03/03/24 at 23:59**. You should submit a single .ipnyb file with your python code and analysis electronically via Canvas.
Please note that this assignment will account for **25 Marks** of your module grade.

## Declaration

By submitting this assignment. I agree to the following:

<font color="red">“I have read and understand the UCC academic policy on plagiarism and I agree to the
requirements set out thereby in relation to plagiarism and referencing. I confirm that I
have referenced and acknowledged properly all sources used in preparation of this
assignment.
I declare that this assignment is entirely my own work based on my personal study. I
further declare that I have not engaged the services of another to either assist me in, or
complete this assignment”</font>

## Specification

The objective of this project is to build a k-Nearest Neighbour algorithm that takes as input training and test dataset and will predict the target binary variable with a reasonable degree of accuracy.

You will find a template for your python code in Canvas along with the dataset. In this project, you must implement your own machine learning library and therefore **you are only allowed to use the following python packages and libraries: NumPy, Pandas.**

This project focuses on the Titanic dataset that we used during the lectures. You can find the dataset at:

https://github.com/andvise/DataAnalyticsDatasets/blob/main/titanic.csv


# Tasks:



# Data Preparation (5 marks)



1.  Load the dataset on Colab


In [1]:
import pandas as pd
df = pd.read_csv("https://github.com/andvise/DataAnalyticsDatasets/blob/dc74027bd063510ede437e4d612f41eb078e49b9/titanic.csv?raw=true")
print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  \
0                             Mr. Owen Harris Braund    male  22.0   
1  Mrs. John Bradley (Florence Briggs Thayer) Cum...  female  38.0   
2                              Miss. Laina Heikkinen  female  26.0   
3        Mrs. Jacques Heath (Lily May Peel) Futrelle  female  35.0   
4                            Mr. William Henry Allen    male  35.0   

   Siblings/Spouses Aboard  Parents/Children Aboard     Fare  
0                        1                      0.0   7.2500  
1                        1                      0.0  71.2833  
2                        0                      0.0   7.9250  
3                        1                      NaN  53.1000  
4                        0                     



2. Display the attributes' name and their data type




In [2]:
df.dtypes

PassengerId                  int64
Survived                     int64
Pclass                       int64
Name                        object
Sex                         object
Age                        float64
Siblings/Spouses Aboard      int64
Parents/Children Aboard    float64
Fare                       float64
dtype: object

3. Delete the columns that should not be included in a k-NN technique. Explain why you removed them in 1-2 sentences.




In [3]:
# Removing columns that are not useful for k-NN
df.drop(['PassengerId', 'Name'], axis=1, inplace=True)
df.dtypes

#PassengerId whas removed because its just an identifier for each row and doesnt hold meaningful information about a passengers survival
#name was also removed , while it might not directly contribute to survival, it could be used to indicate titles or social status but for simiplicity and relevance for K-NN its removed

Survived                     int64
Pclass                       int64
Sex                         object
Age                        float64
Siblings/Spouses Aboard      int64
Parents/Children Aboard    float64
Fare                       float64
dtype: object

4. Replace all missing values with 0


In [4]:
df = df.fillna(0)
print(df.head())

   Survived  Pclass     Sex   Age  Siblings/Spouses Aboard  \
0         0       3    male  22.0                        1   
1         1       1  female  38.0                        1   
2         1       3  female  26.0                        0   
3         1       1  female  35.0                        1   
4         0       3    male  35.0                        0   

   Parents/Children Aboard     Fare  
0                      0.0   7.2500  
1                      0.0  71.2833  
2                      0.0   7.9250  
3                      0.0  53.1000  
4                      0.0   8.0500  


5. Transform the Sex column into a numerical one

In [5]:
# Maps Sex column, male to 0 and female to 1, then convert to int
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1}).astype(int)
print(df.head())

   Survived  Pclass  Sex   Age  Siblings/Spouses Aboard  \
0         0       3    0  22.0                        1   
1         1       1    1  38.0                        1   
2         1       3    1  26.0                        0   
3         1       1    1  35.0                        1   
4         0       3    0  35.0                        0   

   Parents/Children Aboard     Fare  
0                      0.0   7.2500  
1                      0.0  71.2833  
2                      0.0   7.9250  
3                      0.0  53.1000  
4                      0.0   8.0500  


6. Use **Survived** as the target label and the rest of the data frame as features

In [6]:
# Define the features and the target
X = df.drop('Survived', axis=1)  # defines the features, all columns except 'Survived'
y = df['Survived']  # defines the target label 'Survivied'



5. Divide your dataset into 80% for training and 20% for test


In [7]:
# Generates random indices of length of the dataset
import numpy as np
indices = np.random.permutation(len(df))

# 80/20 splits the random indices and assigns them to the train and test data
train_size_80 = int(len(df) * 0.8)
train_indices = indices[:train_size_80]
test_indices = indices[train_size_80:]

# Split the data into training and test sets
X_train_set = X.iloc[train_indices]
y_train_set = y.iloc[train_indices]
X_test_set = X.iloc[test_indices]
y_test_set = y.iloc[test_indices]

6. Scale the columns using min-max scalers


In [8]:
# gets the max and min of the. training set
X_min = X_train_set.min()
X_max = X_train_set.max()

#sclales the train and test set using the formula
X_train_scaled_set = (X_train_set - X_min) / (X_max - X_min)
X_test_scaled_set = (X_test_set - X_min) / (X_max - X_min)


7. Print the shape of the train and test set  

In [9]:
# Print the shape of the manually split train and test sets
print("Training set shape:", X_train_scaled_set.shape)
print("Test set shape:", X_test_scaled_set.shape)


Training set shape: (709, 6)
Test set shape: (178, 6)


# NN implementation (10 Marks)

1. Implement your NN method. To predict **each** point (query point) of the test set, you need to:


> a. Find the training point with the smallest **Euclidian** distance  with the query point


  
> b. Use as a prediction the target value of the point with the smallest distance
to the query point

2. Compute the accuracy

In [10]:
def euclidean_distance(row1, row2):
    """
    Calculates the Euclidean distance between two points.

    Parameters:
    - row1: Numpy array, the first point.
    - row2: Numpy array, the second point.

    Returns:
    - The Euclidean distance between the points.
    """
    return np.sqrt(((row1 - row2) ** 2).sum(axis=1))


In [11]:
def nearest_neighbor(X_train, y_train, X_test):
    """
    Calculates distances to all training points then produces a np.array of predictions using the smallest distance's(NN) target value

    Parameters:
    - X_train: DataFrame, feature set of the scaled training data.
    - y_train: Series, target values of the training data.
    - X_test: DataFrame, feature set of the scaled test data.

    Returns:
    - np.array: Predictions for the test set.
    """
    # Stores the predictions for the test set in an empty list.
    predictions = []

    # Convert to Numpy for faster computation
    X_train_np = X_train.to_numpy()
    X_test_np = X_test.to_numpy()
    y_train_np = y_train.to_numpy()

    # Iterate over each test point
    for test_point in X_test_np:
        # Calculate Euclidean distances from this test point to all training points
        distances = euclidean_distance(X_train_np, test_point)

        # Find the index of the smallest distance (nearest neighbor)
        nearest_index = distances.argmin()

        # Use the target value of the nearest neighbor as the prediction
        predictions.append(y_train_np[nearest_index])

    return np.array(predictions)


In [12]:
# Makes predictions for the test set
y_pred = nearest_neighbor(X_train_scaled_set, y_train_set, X_test_scaled_set)
y_pred

array([0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1])

In [13]:
def calculate_accuracy(y_true, y_pred):
    """
    Accuracy function that compares predicted values to true values

    Parameters:
    - y_true: Series, actual target values.
    - y_pred: np.array, predicted target values.

    Returns:
    - float: The accuracy of the predictions.
    """
    #compares predictions with the true test labels
    correct_predictions = (y_true == y_pred).sum()
    accuracy = correct_predictions / len(y_true)
    return accuracy


In [14]:
# Calculates the accuracy of the NN model
accuracy = calculate_accuracy(y_test_set, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7640449438202247


# k-NN implementation (10 Marks)

1. Extend the previous implementation by using the average of the k closest neighbours as a prediction

2. Add the **Manhattan** and **Hamming** distance.


2. Compute the accuracy

> a. Is it more accurate compared to the previous implementation?

In [15]:
def manhattan_distance(row1, row2):
    """
    Calculate the Manhattan distance between two points.

    Parameters:
    - row1: Numpy array, the first point.
    - row2: Numpy array, the second point.

    Returns:
    - The Manhattan distance between the points.
    """
    return np.abs(row1 - row2).sum(axis=1)

In [16]:
def hamming_distance(row1, row2):
    """
    Calculate the Hamming distance between two points.

    Parameters:
    - row1: Numpy array, the first point.
    - row2: Numpy array, the second point.

    Returns:
    - The Hamming distance between the points.
    """
    return (row1 != row2).astype(int).sum(axis=1)

In [17]:
def k_nearest_neighbors(X_train, y_train, X_test, k, distance_fn):
    """
    Predict the target value for each test point based on the k nearest neighbors in the training set.

    Parameters:
    - X_train: DataFrame, feature set of the training data.
    - y_train: Series, target values of the training data.
    - X_test: DataFrame, feature set of the test data.
    - k: int, number of nearest neighbors to consider.
    - distance_fn: function, the distance function to use.

    Returns:
    - np.array: Predictions for the test set.
    """

    # Stores the predictions for the test set in an empty list.
    predictions = []

    #Convert to Numpy for faster computation
    X_train_np = X_train.to_numpy()
    X_test_np = X_test.to_numpy()
    y_train_np = y_train.to_numpy()

    # Iterate over each test point
    for test_point in X_test_np:
        #calculate the specified distance measure from this test point to all training points
        distances = distance_fn(X_train_np, test_point)

        # Gets the indices of the k nearest neighbours
        nearest_indices = distances.argsort()[:k]

        # Gets the corresponding target values for these indices
        nearest_labels = y_train_np[nearest_indices]

        # Determines the most common label (class) among the nearest neighbours
        prediction = np.bincount(nearest_labels).argmax()

        # Appends the prediction to the predictions list
        predictions.append(prediction)

    return np.array(predictions)


3. Test your approach for k  values = [1, 3, 5, 7] and the three distance measures

> a. Which is the best combination?

> b. How can you improve the results?


In [18]:
# Define the k values and distance measures
k_values = [1, 3, 5, 7]
distance_measures = [euclidean_distance, manhattan_distance, hamming_distance]

# Iterate over each distance measure
for distance in distance_measures:
    distance_name = distance.__name__  # Gets the name of the function for printing clarity
    # Iterate over each k value
    for k in k_values:
        # Compute predictions using k-NN
        y_pred_knn = k_nearest_neighbors(X_train_scaled_set, y_train_set, X_test_scaled_set, k, distance)

        # Compute accuracy
        accuracy_knn = calculate_accuracy(y_test_set, y_pred_knn)

        # Print accuracies for comparison
        print(f"Accuracy = {accuracy_knn}, when k value = {k} using {distance_name} measure")


Accuracy = 0.7752808988764045, when k value = 1 using euclidean_distance measure
Accuracy = 0.8314606741573034, when k value = 3 using euclidean_distance measure
Accuracy = 0.8370786516853933, when k value = 5 using euclidean_distance measure
Accuracy = 0.8539325842696629, when k value = 7 using euclidean_distance measure
Accuracy = 0.7752808988764045, when k value = 1 using manhattan_distance measure
Accuracy = 0.848314606741573, when k value = 3 using manhattan_distance measure
Accuracy = 0.848314606741573, when k value = 5 using manhattan_distance measure
Accuracy = 0.8651685393258427, when k value = 7 using manhattan_distance measure
Accuracy = 0.8146067415730337, when k value = 1 using hamming_distance measure
Accuracy = 0.8202247191011236, when k value = 3 using hamming_distance measure
Accuracy = 0.8146067415730337, when k value = 5 using hamming_distance measure
Accuracy = 0.8146067415730337, when k value = 7 using hamming_distance measure


## Question 2a, Is it more accurate compared to the previous implementation?
Yes the KNN model is more accurate than the previous NN model across all combinations, the previous implementation of the NN model returned an Accuracy: 0.7640449438202247, however every iteration of the KNN model output a higher accuracy with the lowest Accuracy = 0.7752808988764045, when k value = 1 using euclidean_distance measure.

## Question 3a, Which is the best combination?
When k value = 7 using manhattan_distance measure, this gave the highest Accuracy = 0.8651685393258427

 however other combinations with varying K using Manattan and Euclidiean also showed improved accuracy in different instances, this could be due to the random splitting of the data sets being favoured by other combinations.

## Question 3b, How can you improve the results?
To improve results the following approaches could be taken:
1. Feature engineering: transforming features could give more meaningful inputs into the model, one feature that could be transformed are the names, instead of excluding this data from the model, class and titles could be be extracted and new features based on combined attributes could provide new insights and improve results.

2. Data preprocessing: Further analysing the preprocessing steps could provide more meaningful data to improve the models results, this could involve using different methods to handle missing data values, scaling, normalisation standardisation techniques as these have a significant impact on KNN results.
Properly encoding categorical variables.

3. Parameter tuning: Using different values of k differing to the tested ones, this could yield improved results, a weighted KNN could also improve results with closer points having more influence on the predictions

4. Distance Measure: other measures could improve results, such as Minkowski or Cosine [1].

5. Cross Validation: Using cross validation could improve results by evaluating the models performance better and ensures the model generalises well across difference subsets of the data [1].

6. Address imbalance data: if data is unbalanced,oversampling the minority class and undersampling the majority class can be used, however the target variable is relatively equally distributed with  a 61:39 split which means its reasonably balanced  Survived: 0    0.609309,
1    0.390691 [1].

7. Ensemble techniques: combining multiple models can yield better results than from a single KNN model [1].

8. Dimensionality Reduction: using techniques such as PCA principical component analysis where dimensionality is reduced without losing a lot of information which can prevent the curse of Dimensionality and prevent overfitting [1].

Reference: [1] btd. (2023). "Strategies for k-NN Classifier Fine-Tuning". [*Medium.com*]( https://baotramduong.medium.com/strategies-for-knn-classifier-fine-tuning-f225b873da4a )




**Reminder: You can not use any library, with the exception of pandas and NumPy. So scikit-learn is forbidden!**

## Marking Scheme
**Program Correctness:** Your program should work correctly on all inputs (including new datasets). Also, if there are any specifications about how the program should be written, or how the output should appear, those specifications should be followed.

**Readability:** Variables functions should have meaningful names. The code should be organised into functions/methods where appropriate. There should be an appropriate amount of white space so that the code is readable, and the indentation should be consistent.

**Documentation:** your code and functions/methods should be appropriately commented. However, not every line should be commented because that makes your code overly busy. Think carefully about where comments are added.


**Code Elegance:** There are many ways to write the same functionality into your code, and some of them are needlessly slow or complicated. For example, if you are repeating the same code, it should be inside creating a new method/function or for loop

**Code efficiency:** The implementation is logically well-designed without inappropriate
design choices (e.g., unnecessary loops).